In [1]:
import pandas as pd
import numpy as np

# подготовка очищенного датасета на базе обработки в HW_6_baseline
from src.data.clean_data import clean_data
from src.data.feature_combinations import feature_combinations

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score

# Чистим данные и добавляем "секретные" признаки в датасет

In [2]:
dwarves = pd.read_csv("../data/interim/dwarves_secret_info.csv")
dwarves.head()

,Secret_dwarf_info_1,Secret_dwarf_info_2,Secret_dwarf_info_3,Successful_deals_count,Tavern,Hashed_deal_detail_1,Hashed_deal_detail_2,Hashed_deal_detail_3,Hashed_deal_detail_4,Hashed_deal_detail_5,...,Tavern_district_2,Tavern_district_3,Tavern_district_4,Tavern_district_5,Tavern_district_6,Tavern_district_7,day_before_first_defolt,Deal_day,Deal_month,is_weekend
0,3.5,-2.0,5.0,0.0,7,2.5,-3,8,2.5,-3,...,0,1,0,0,0,0,68,5,11,0
1,3.5,-2.0,5.0,2.0,7,2.5,-3,14,3.5,-3,...,0,0,1,0,0,0,222,26,8,0
2,3.5,-2.0,5.0,0.0,7,2.5,-3,8,2.5,-3,...,0,0,0,0,1,0,99,18,2,0
3,3.5,-2.0,5.0,0.0,13,2.5,-2,5,2.5,-3,...,1,0,0,0,0,0,39,30,4,1
4,3.5,-2.0,5.0,0.0,39,2.5,-3,7,2.5,-3,...,0,0,1,0,0,0,60,19,9,0


In [3]:
feature_combinations(dwarves)

# Подготавливаем данные для линейных моделей

In [4]:
X = dwarves.drop(columns=['Default'])
y = dwarves['Default']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, stratify=y, random_state=2023)

In [5]:
rf_clf = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=1000))

rf_clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(n_estimators=1000))])

In [363]:
# SVM с ядром rbf
svc_clf = make_pipeline(StandardScaler(), SVC(kernel="rbf", degree=2, C=50, random_state=2023))

svc_clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=50, degree=2, random_state=2023))])

In [364]:
y_score = svc_clf.decision_function(X_test)
y_score = np.where(y_score > -0.9, 1, 0)
roc_auc_score(y_test, y_score)

0.5980113636363635

In [6]:
y_score = rf_clf.predict_proba(X_test)
y_score = np.where(y_score[:,1] > 0.108, 1, 0)
roc_auc_score(y_test, y_score)

0.8571428571428572

# Готовимся к сдаче на тестовом датасете

- готовим базовый test датасет по тому же Pipeline-у,
- обучаем модель на всех тренировочных данных
- выставляем посчитанный threshold
- считаем предсказания
- выгружаем в csv

# SVC

In [28]:
# преобразуем test датасет для предсказаний
dwarves_test_lms = clean_data(path="../data/raw/test.csv", secret=True)

In [35]:
y_score = svc_clf.decision_function(X_test)
y_std = (y_score - y_score.min()) / (y_score.max() - y_score.min())

In [36]:
prediction = pd.DataFrame(y_std, columns=['Prediction'])

Собираем предсказания в csv

In [37]:
# датасет к сдаче в LMS
test_df = pd.read_csv("../data/raw/test.csv")
prediction_lms= test_df[['Deal_id']]
prediction_lms['Prediction'] = prediction
prediction_lms.head()

C:\Users\Dmitry\AppData\Local\Temp\ipykernel_5368\2310115546.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction_lms['Prediction'] = prediction


,Deal_id,Prediction
0,72875713,0.119675
1,75825544,0.048682
2,81809181,0.227181
3,87083256,0.273834
4,84651519,0.237323


In [357]:
prediction_lms.to_csv("../data/processed/prediction_SVC_rbf_2_50.csv", index=False)

In [ ]:
# 0.625 svc_clf = make_pipeline(StandardScaler(), SVC(kernel="rbf", degree=2, C=50, random_state=2023))
# хуже catboost при перебираемых 'degree' и 'C'.

# RandomForest

In [7]:
# преобразуем test датасет для предсказаний
dwarves_test_lms = clean_data(path="../data/raw/test.csv", secret=True)

In [8]:
# добавим фичи
feature_combinations(dwarves_test_lms)

In [9]:
y_score = rf_clf.predict_proba(dwarves_test_lms)[:,1]
# ручной аналог min-max scaler поверх вероятностей
y_std = (y_score - y_score.min()) / (y_score.max() - y_score.min())
prediction = pd.DataFrame(y_std, columns=['Prediction'])

In [12]:
# датасет к сдаче в LMS
test_df = pd.read_csv("../data/raw/test.csv")
prediction_lms= test_df[['Deal_id']]
prediction_lms['Prediction'] = prediction
prediction_lms.head()

C:\Users\Dmitry\AppData\Local\Temp\ipykernel_7028\2310115546.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction_lms['Prediction'] = prediction


,Deal_id,Prediction
0,72875713,0.069597
1,75825544,0.058608
2,81809181,0.302198
3,87083256,0.326007
4,84651519,0.150183


In [13]:
prediction_lms.to_csv("../data/processed/prediction_RF_1000_add_features_001.csv", index=False)

In [14]:
# 0.692 без фичей
# 0.698  с фичами
# 0.730 - на всем датасете. Сдано, по факту вышло без подбора гиперпараметров случайного леса.

# Выводы и что можно улучшить?

- возможно подбор гиперпараметров Random Forest смог бы еще поднять ROC-AUC.
- решение о заполнении secret-фич медианой, лучшее что удалось найти, но не факт что это так. Они оказались полезными для моделей, поэтому их сбор крайне важен.
- На основе feature_importances_ можно скорректировать собираемые фичи для будущего датасета.
- Не лишним было бы знание о дате последнего дефолта, помимо первого, он бы дал знание о том, сколько визитов было без дефолта и сколько при этом прошло времени. А также общее количество дефолтов у каждого гнома.
- засекреченные признаки по заказу ('Hashed_deal_detail_n') и их комбинации (перемножением) практически не вносили вклад в итоговый результат.
- пропустить отнормированные данные без комбинации фичей через полносвязный многослойный перспетрон. Или трансформер, а с помощью латентного представления решать задачу классификации.